In [1]:
import os
import numpy as np 

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

from time import time

cuda = torch.cuda.is_available()

### Load Data

In [2]:
RAW_DATA_DIR = "../Data/Raw/"
PROCESSED_DATA_DIR = "../Data/Processed/"

class CovariateDataset(Dataset):
    def __init__(self, file_name_pattern, file_name_args):
        self.file_name = file_name_pattern.format(*file_name_args)
        self.data = np.loadtxt(RAW_DATA_DIR + self.file_name + ".csv", delimiter=",")[:, 1:] # remove bias
        
    def __getitem__(self, index):
        return (self.data[index].astype(float), 0)

    def __len__(self):
        return self.data.shape[0]
    
    def save_processed_data(self, data, loss):
        name = PROCESSED_DATA_DIR + self.file_name+"_{}.csv".format(loss)
        np.savetxt(name, data, delimiter=",")

### Define Model

In [3]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(10, 128),
            nn.ReLU(True),
            nn.Linear(128, 4))
        self.decoder = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(True),
            nn.Linear(128, 10))

    def forward(self, x):
        encoded_values = self.encoder(x)
        x = self.decoder(encoded_values)
        return x, encoded_values

### Train and Process Utils

In [4]:
def reconstruction_sparsity_loss(output, target, encoded_values):
        sparsity_scalar = Variable(torch.FloatTensor([0.0005]))
        if cuda:
            sparsity_scalar = sparsity_scalar.cuda()
            
        mse_loss = nn.MSELoss()
        reconstruction_loss = mse_loss(output, target)
        sparsity_loss = encoded_values.abs().sum()*sparsity_scalar
        return reconstruction_loss + sparsity_loss 
    
def reconstruction_loss(output, target, encoded_values):
    mse_loss = nn.MSELoss()
    reconstruction_loss = mse_loss(output, target)
    return reconstruction_loss

In [5]:
loss_functions = ["reconstruction", "sparsity"]

def train_model(model_class, dataset, loss="reconstruction", verbose=True):
    model = model_class()
    if cuda:
        model = model.cuda()

    num_epochs = 10000
    batch_size = 1000
    learning_rate = 1e-1
    lr_sched = True
    
    
    if loss == loss_functions[0]:
        criterion = reconstruction_loss
    elif loss == loss_functions[1]:
        criterion = reconstruction_sparsity_loss
         
    optimizer = torch.optim.Adam(
        model.parameters(), lr=learning_rate, weight_decay=1e-5)

    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2000, 5000], gamma=0.1)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    final_loss = None
    
    for epoch in range(num_epochs):
        if lr_sched:
            scheduler.step()

        for data in dataloader:
            data_batch, _ = data
            data_batch = Variable(data_batch)
            data_batch = data_batch.float()

            if cuda:
                data_batch = data_batch.cuda()

            # Forward pass
            output, encoded_values = model(data_batch)

            loss = criterion(output, data_batch, encoded_values)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        if epoch%int(num_epochs/10) == int(num_epochs/10)-1 and verbose:
            print('epoch [{}/{}], loss:{:.4f}'
                  .format(epoch + 1, num_epochs, loss.data[0]))
        
        if epoch == (num_epochs-1):
            final_loss = loss.data[0]
            print("Final loss: loss:{:.4f}".format(final_loss))

    torch.save(model.state_dict(), "../Models/simple_autoencoder_{}.pth".format(dataset_number))
    return model, final_loss

def encode_data(model, dataset, loss):
    all_data = torch.from_numpy(dataset.data)
    all_data = Variable(all_data)
    all_data = all_data.float()
    
    if cuda:
        all_data = all_data.cuda()

    output = model.encoder(all_data)
    
    if cuda:
        output = output.cpu()
        
    dataset.save_processed_data(output.data.numpy(), loss)
    return output.data.numpy()

In [6]:
# dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, "A_add_lin", 1])
# trained_model, final_loss = train_model(
#                                     autoencoder,
#                                     dataset,
#                                     loss="sparsity",
#                                     verbose=True)
# encode_data(trained_model, dataset, "sparsity")

### Train and Encode

In [ ]:
models_to_rerun = []
datasets_to_process = range(50, 200)
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

for dataset_number in datasets_to_process:
    print("Starting run for Dataset {}".format(dataset_number))
    
    for model_name in assignment_model_names:
        print("-- Running for model name: ", model_name)
        
        for loss_type in loss_functions:
            print("---- Running for loss: ", loss_type)
            
            start = time()
            
            dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, model_name, dataset_number])
            trained_model, final_loss = train_model(
                                                autoencoder,
                                                dataset,
                                                loss=loss_type,
                                                verbose=True)
            encode_data(trained_model, dataset, loss=loss_type)
            
            print("---- Done in ", time() - start, " seconds\n")
            
            # Catch bad runs
            if loss_type == loss_functions[0] and final_loss > 0.30:
                models_to_rerun.append((model_name, dataset_number, loss_type))
            elif loss_type == loss_functions[1] and final_loss > 1.0:
                models_to_rerun.append((model_name, dataset_number, loss_type))
                
    print("================\n\n")
    
print("Rerun: ", models_to_rerun)

Starting run for Dataset 50
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1733
epoch [2000/10000], loss:0.1567
epoch [3000/10000], loss:0.1512
epoch [4000/10000], loss:0.1487
epoch [5000/10000], loss:0.1451
epoch [6000/10000], loss:0.1442
epoch [7000/10000], loss:0.1434
epoch [8000/10000], loss:0.1418
epoch [9000/10000], loss:0.1402
epoch [10000/10000], loss:0.1386
Final loss: loss:0.1386
---- Done in  62.499022006988525  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5429
epoch [2000/10000], loss:0.5391
epoch [3000/10000], loss:0.5229
epoch [4000/10000], loss:0.4270
epoch [5000/10000], loss:0.4223
epoch [6000/10000], loss:0.4196
epoch [7000/10000], loss:0.4191
epoch [8000/10000], loss:0.4178
epoch [9000/10000], loss:0.3346
epoch [10000/10000], loss:0.3273
Final loss: loss:0.3273
---- Done in  61.60608243942261  seconds

-- Running for model name:  B_add_mild_nlin
---- Running for loss:  reconstruction
epoc

epoch [2000/10000], loss:0.1770
epoch [3000/10000], loss:0.1725
epoch [4000/10000], loss:0.1694
epoch [5000/10000], loss:0.1653
epoch [6000/10000], loss:0.1642
epoch [7000/10000], loss:0.1635
epoch [8000/10000], loss:0.1622
epoch [9000/10000], loss:0.1611
epoch [10000/10000], loss:0.1594
Final loss: loss:0.1594
---- Done in  60.03660559654236  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.4378
epoch [2000/10000], loss:0.3944
epoch [3000/10000], loss:0.2585
epoch [4000/10000], loss:0.2399
epoch [5000/10000], loss:0.2335
epoch [6000/10000], loss:0.2327
epoch [7000/10000], loss:0.2318
epoch [8000/10000], loss:0.2306
epoch [9000/10000], loss:0.2291
epoch [10000/10000], loss:0.2277
Final loss: loss:0.2277
---- Done in  62.038952112197876  seconds

-- Running for model name:  D_mild_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1864
epoch [2000/10000], loss:0.1774
epoch [3000/10000], loss:0.1594
epoch [4000/10000], loss:0.1542
epoch [5000/1

epoch [6000/10000], loss:0.1408
epoch [7000/10000], loss:0.1403
epoch [8000/10000], loss:0.1395
epoch [9000/10000], loss:0.1383
epoch [10000/10000], loss:0.1357
Final loss: loss:0.1357
---- Done in  59.612022399902344  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5044
epoch [2000/10000], loss:0.5257
epoch [3000/10000], loss:0.5036
epoch [4000/10000], loss:0.5010
epoch [5000/10000], loss:0.4089
epoch [6000/10000], loss:0.4067
epoch [7000/10000], loss:0.3263
epoch [8000/10000], loss:0.2379
epoch [9000/10000], loss:0.2322
epoch [10000/10000], loss:0.2277
Final loss: loss:0.2277
---- Done in  62.21431374549866  seconds

-- Running for model name:  F_mod_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1878
epoch [2000/10000], loss:0.1739
epoch [3000/10000], loss:0.1690
epoch [4000/10000], loss:0.1666
epoch [5000/10000], loss:0.1633
epoch [6000/10000], loss:0.1625
epoch [7000/10000], loss:0.1617
epoch [8000/10000], loss:0.1604
epoch [9000/10

epoch [10000/10000], loss:0.1497
Final loss: loss:0.1497
---- Done in  59.94474673271179  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.4291
epoch [2000/10000], loss:0.3614
epoch [3000/10000], loss:0.3223
epoch [4000/10000], loss:0.3145
epoch [5000/10000], loss:0.3134
epoch [6000/10000], loss:0.3103
epoch [7000/10000], loss:0.3097
epoch [8000/10000], loss:0.3090
epoch [9000/10000], loss:0.3081
epoch [10000/10000], loss:0.3073
Final loss: loss:0.3073
---- Done in  62.29619812965393  seconds



Starting run for Dataset 54
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1879
epoch [2000/10000], loss:0.1673
epoch [3000/10000], loss:0.1511
epoch [4000/10000], loss:0.1471
epoch [5000/10000], loss:0.1434
epoch [6000/10000], loss:0.1423
epoch [7000/10000], loss:0.1417
epoch [8000/10000], loss:0.1410
epoch [9000/10000], loss:0.1403
epoch [10000/10000], loss:0.1387
Final loss: loss:0.1387
---- Done in  59.963058471679

epoch [1000/10000], loss:0.4694
epoch [2000/10000], loss:0.5046
epoch [3000/10000], loss:0.4168
epoch [4000/10000], loss:0.4130
epoch [5000/10000], loss:0.4153
epoch [6000/10000], loss:0.4101
epoch [7000/10000], loss:0.4097
epoch [8000/10000], loss:0.4093
epoch [9000/10000], loss:0.4087
epoch [10000/10000], loss:0.4080
Final loss: loss:0.4080
---- Done in  62.653647661209106  seconds

-- Running for model name:  C_add_mod_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.2025
epoch [2000/10000], loss:0.1835
epoch [3000/10000], loss:0.1704
epoch [4000/10000], loss:0.1628
epoch [5000/10000], loss:0.1577
epoch [6000/10000], loss:0.1567
epoch [7000/10000], loss:0.1558
epoch [8000/10000], loss:0.1540
epoch [9000/10000], loss:0.1508
epoch [10000/10000], loss:0.1485
Final loss: loss:0.1485
---- Done in  60.11922597885132  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5977
epoch [2000/10000], loss:0.6624
epoch [3000/10000], loss:0.6413
epoch [4000/10

...

epoch [6000/10000], loss:0.1542
epoch [7000/10000], loss:0.1538
epoch [8000/10000], loss:0.1526
epoch [9000/10000], loss:0.1514
epoch [10000/10000], loss:0.1497
Final loss: loss:0.1497
---- Done in  60.164220571517944  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5491
epoch [2000/10000], loss:0.5593
epoch [3000/10000], loss:0.5171
epoch [4000/10000], loss:0.4234
epoch [5000/10000], loss:0.3272
epoch [6000/10000], loss:0.3248
epoch [7000/10000], loss:0.3235
epoch [8000/10000], loss:0.3222
epoch [9000/10000], loss:0.3209
epoch [10000/10000], loss:0.3199
Final loss: loss:0.3199
---- Done in  63.10818934440613  seconds

-- Running for model name:  D_mild_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1976
epoch [2000/10000], loss:0.2464
epoch [3000/10000], loss:0.1580
epoch [4000/10000], loss:0.1513
epoch [5000/10000], loss:0.1468
epoch [6000/10000], loss:0.1459
epoch [7000/10000], loss:0.1451
epoch [8000/10000], loss:0.1443
epoch [9000/1

epoch [1000/10000], loss:0.5963
epoch [2000/10000], loss:0.4545
epoch [3000/10000], loss:0.4347
epoch [4000/10000], loss:0.4293
epoch [5000/10000], loss:0.4279
epoch [6000/10000], loss:0.4260
epoch [7000/10000], loss:0.4254
epoch [8000/10000], loss:0.4242
epoch [9000/10000], loss:0.4196
epoch [10000/10000], loss:0.3383
Final loss: loss:0.3383
---- Done in  64.4527199268341  seconds

-- Running for model name:  F_mod_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1727
epoch [2000/10000], loss:0.1534
epoch [3000/10000], loss:0.1452
epoch [4000/10000], loss:0.1419
epoch [5000/10000], loss:0.1392
epoch [6000/10000], loss:0.1386
epoch [7000/10000], loss:0.1382
epoch [8000/10000], loss:0.1377
epoch [9000/10000], loss:0.1368
epoch [10000/10000], loss:0.1359
Final loss: loss:0.1359
---- Done in  64.23483228683472  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.4180
epoch [2000/10000], loss:0.3873
epoch [3000/10000], loss:0.3288
epoch [4000/1000

epoch [6000/10000], loss:0.5169
epoch [7000/10000], loss:0.5161
epoch [8000/10000], loss:0.5150
epoch [9000/10000], loss:0.4384
epoch [10000/10000], loss:0.4317
Final loss: loss:0.4317
---- Done in  62.75292158126831  seconds



Starting run for Dataset 71
-- Running for model name:  A_add_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1739
epoch [2000/10000], loss:0.1634
epoch [3000/10000], loss:0.1544
epoch [4000/10000], loss:0.1516
epoch [5000/10000], loss:0.1483
epoch [6000/10000], loss:0.1471
epoch [7000/10000], loss:0.1461
epoch [8000/10000], loss:0.1453
epoch [9000/10000], loss:0.1437
epoch [10000/10000], loss:0.1424
Final loss: loss:0.1424
---- Done in  60.20402455329895  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5576
epoch [2000/10000], loss:0.5458
epoch [3000/10000], loss:0.5247
epoch [4000/10000], loss:0.5234
epoch [5000/10000], loss:0.5225
epoch [6000/10000], loss:0.5204
epoch [7000/10000], loss:0.5202
epoch [8000/10000], lo

epoch [9000/10000], loss:0.5147
epoch [10000/10000], loss:0.5145
Final loss: loss:0.5145
---- Done in  62.631312131881714  seconds

-- Running for model name:  C_add_mod_nlin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1766
epoch [2000/10000], loss:0.1488
epoch [3000/10000], loss:0.1416
epoch [4000/10000], loss:0.1379
epoch [5000/10000], loss:0.1338
epoch [6000/10000], loss:0.1328
epoch [7000/10000], loss:0.1321
epoch [8000/10000], loss:0.1309
epoch [9000/10000], loss:0.1294
epoch [10000/10000], loss:0.1274
Final loss: loss:0.1274
---- Done in  64.46986865997314  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.5331
epoch [2000/10000], loss:0.4600
epoch [3000/10000], loss:0.4277
epoch [4000/10000], loss:0.4224
epoch [5000/10000], loss:0.4211
epoch [6000/10000], loss:0.4190
epoch [7000/10000], loss:0.4187
epoch [8000/10000], loss:0.4183
epoch [9000/10000], loss:0.4178
epoch [10000/10000], loss:0.4173
Final loss: loss:0.4173
---- Done in  67.2683

epoch [1000/10000], loss:0.1827
epoch [2000/10000], loss:0.1577
epoch [3000/10000], loss:0.1495
epoch [4000/10000], loss:0.1471
epoch [5000/10000], loss:0.1442
epoch [6000/10000], loss:0.1437
epoch [7000/10000], loss:0.1430
epoch [8000/10000], loss:0.1421
epoch [9000/10000], loss:0.1410
epoch [10000/10000], loss:0.1397
Final loss: loss:0.1397
---- Done in  60.81527781486511  seconds

---- Running for loss:  sparsity
epoch [1000/10000], loss:0.3961
epoch [2000/10000], loss:0.4512
epoch [3000/10000], loss:0.3343
epoch [4000/10000], loss:0.3274
epoch [5000/10000], loss:0.3246
epoch [6000/10000], loss:0.3231
epoch [7000/10000], loss:0.3225
epoch [8000/10000], loss:0.3218
epoch [9000/10000], loss:0.3208
epoch [10000/10000], loss:0.3199
Final loss: loss:0.3199
---- Done in  63.783880949020386  seconds

-- Running for model name:  F_mod_nadd_lin
---- Running for loss:  reconstruction
epoch [1000/10000], loss:0.1809
epoch [2000/10000], loss:0.1706
epoch [3000/10000], loss:0.1658
epoch [4000/10

In [8]:
# models_to_rerun = [('A_add_lin', 12, 'sparsity'), ('G_mod_nadd_mod_nlin', 40, 'sparsity')]

for model_name, dataset_number, loss_type in models_to_rerun:
    dataset = CovariateDataset("n_{}_model_{}_v_{}_covar_data", [1000, model_name, dataset_number])
    trained_model, final_loss = train_model(
                                        autoencoder,
                                        dataset,
                                        loss=loss_type,
                                        verbose=True)
    encode_data(trained_model, dataset, loss=loss_type)

epoch [1000/10000], loss:0.4875
epoch [2000/10000], loss:0.4456
epoch [3000/10000], loss:0.4104
epoch [4000/10000], loss:0.3195
epoch [5000/10000], loss:0.3145
epoch [6000/10000], loss:0.3127
epoch [7000/10000], loss:0.3118
epoch [8000/10000], loss:0.3107
epoch [9000/10000], loss:0.3096
epoch [10000/10000], loss:0.3084
Final loss: loss:0.3084
epoch [1000/10000], loss:0.5869
epoch [2000/10000], loss:0.5415
epoch [3000/10000], loss:0.5173
epoch [4000/10000], loss:0.4197
epoch [5000/10000], loss:0.4216
epoch [6000/10000], loss:0.4136
epoch [7000/10000], loss:0.4132
epoch [8000/10000], loss:0.4127
epoch [9000/10000], loss:0.4123
epoch [10000/10000], loss:0.4118
Final loss: loss:0.4118
